# [DE:Betretungsverbote für Gebiete im Winter](https://wiki.openstreetmap.org/wiki/DE:Betretungsverbote_für_Gebiete_im_Winter)

* Extracts all ways via Overpass and filter/analyze further based on the definition above
* perform some basic analyis (multiple types, untyped)
* create a GeoJSON with feature properties set to type<1-8> for direct rendering
  * style properties are set for https://geojson.io

## ToDo:

* support for relations
* adjust style properties for types (within limits of both https://geojson.io and leaflet)
* visualize geojson in Notebook
* direct import for geojson.io via API?


## Overpass Basis-Query

In [1]:
import sys, re, os
# https://github.com/jazzband/geojson
import geojson
# https://python-overpy.readthedocs.io/en/latest/index.html
import overpy
from IPython.core.display import display, HTML

silent = False
if sys.argv[1] == "silent":
    silent = True

api = overpy.Overpass()

# http://norbertrenner.de/osm/bbox.html
bbox = "47.378,11.078,47.768,13.111"
# [bbox:{bbox}];
query = f"""
(
way["boundary"="protected_area"]["protect_class"="14"];
relation["boundary"="protected_area"]["protect_class"="14"];
);
(._;>;);
out body;
"""

result = api.query(query)

allResults = []         # this will be a list of relations and ways which are not part of a relation - i.e. the list we want to check
allRelationWayIds = []  # list of wayIds which are part of a relation (which can be skipped form result.ways here)

for rel in result.relations:
    allResults.append(rel)
    for member in rel.members:
        if (type(member) == overpy.RelationWay):
            way = result.get_way(member.ref, resolve_missing=True)
            allRelationWayIds.append(way.id)

for way in result.ways:
    if not way.id in allRelationWayIds:
        allResults.append(way)

if not silent:
    print (f"retrieved {len(allResults)} ways and relations")

retrieved 319 ways and relations


In [2]:
# transform all tags into the names used in the "SQL-like" queries in the Wiki
def get_tags(tags):
    seasonal = tags.get("seasonal")
    ski = tags.get("ski")
    ski_conditional = tags.get("ski:conditional") or ""
    access = tags.get("access")
    access_conditional = tags.get("access:conditional") or ""
    access_offroad = tags.get("access:offroad") or ""
    access_offroad_conditional = tags.get("access:offroad:conditional") or ""
    
    return seasonal, ski, ski_conditional, access, access_conditional, access_offroad, access_offroad_conditional

## 1 - [Ganzjähriges Befahrungsverbot für Skifahrer](https://wiki.openstreetmap.org/wiki/DE:Betretungsverbote_f%C3%BCr_Gebiete_im_Winter#Ganzj.C3.A4hriges_Befahrungsverbot_f.C3.BCr_Skifahrer)


In [3]:
# Note: in a non-notebook script, all queries should be done in a single loop
allTypes= [] # this will be a list[0-7][], i.e. 8 lists, grouped by types
wayRels = []
for wayRel in allResults:
    seasonal, ski, ski_conditional, access, access_conditional, access_offroad, access_offroad_conditional = get_tags(wayRel.tags)
    
    if (
        seasonal is None
        or seasonal == 'no'
    ) and (
        ski in ['no', 'private'] or (
            access in ['no','private'] and (
                ski is None
                or not ski in ['yes','designated','permissive']
            )
        )
    ):
        wayRels.append(wayRel)
if not silent:        
    print(f"type1 = {len(wayRels)}")
allTypes.append(wayRels)

type1 = 30


## 2 - [Monatsabhängiges oder saisonales Befahrungsverbot für Skifahrer](https://wiki.openstreetmap.org/wiki/DE:Betretungsverbote_f%C3%BCr_Gebiete_im_Winter#Monatsabh.C3.A4ngiges_oder_saisonales_Befahrungsverbot_f.C3.BCr_Skifahrer)

In [4]:
wayRels = []
for wayRel in allResults:
    seasonal, ski, ski_conditional, access, access_conditional, access_offroad, access_offroad_conditional = get_tags(wayRel.tags)

    if (
        (
            (
                not (seasonal is None or seasonal == 'no')
            ) and (
                ski in ['no', 'private']
                or re.match('(no|private) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', ski_conditional)
                or (
                    (
                        access in ['no','private']
                        or re.match('(no|private) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', access_conditional)
                    ) and (
                        ski is None
                        or not ski in ['yes','designated','permissive']
                    )
                )
            )
            ) or (
                re.match('(no|private) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', ski_conditional)
                or (
                    (
                        re.match('(no|private) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', access_conditional)
                    ) and (
                        ski is None
                    )
                )
            )
        ):
        wayRels.append(wayRel)
if not silent:        
    print(f"type2 = {len(wayRels)}")        
allTypes.append(wayRels)

type2 = 12


## 3 - [Ganzjährige "Bitte um Nichtbefahrung" für Skifahrer](https://wiki.openstreetmap.org/wiki/DE:Betretungsverbote_f%C3%BCr_Gebiete_im_Winter#Ganzj.C3.A4hrige_.22Bitte_um_Nichtbefahrung.22_f.C3.BCr_Skifahrer)

In [5]:
wayRels = []
for wayRel in allResults:
    seasonal, ski, ski_conditional, access, access_conditional, access_offroad, access_offroad_conditional = get_tags(wayRel.tags)
    
    if ( 
        seasonal is None
        or seasonal == 'no'
    ) and (
        ski == 'discouraged'
        or (
            access == 'discouraged' and (
                ski is None
                or not ski in ['yes','designated','permissive']
            )
        )
    ):
        wayRels.append(wayRel)
if not silent:
    print(f"type3 = {len(wayRels)}")
allTypes.append(wayRels)

type3 = 0


## 4 - [Monatsabhängige oder saisonale "Bitte um Nichtbefahrung" für Skifahrer](https://wiki.openstreetmap.org/wiki/DE:Betretungsverbote_f%C3%BCr_Gebiete_im_Winter#Monatsabh.C3.A4ngige_oder_saisonale_.22Bitte_um_Nichtbefahrung.22_f.C3.BCr_Skifahrer)

In [6]:
wayRels = []
for wayRel in allResults:
    seasonal, ski, ski_conditional, access, access_conditional, access_offroad, access_offroad_conditional = get_tags(wayRel.tags)

    if (
        (
            (
                not (seasonal is None or seasonal == 'no') 
            ) and (
                ski == 'discouraged'
                or re.match('(discouraged) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', ski_conditional)
                or (
                    (
                        access == 'discouraged' 
                        or re.match('(discouraged) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', access_conditional)
                    ) and (
                        ski is None
                        or not ski in ['yes','designated','permissive']
                    )
                )
            )
        ) or (
            re.match('(discouraged) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', ski_conditional)
            or (
                (
                    re.match('(discouraged) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', access_conditional)
                ) and (
                    ski is None
                )
            )        
        )
    ):
        wayRels.append(wayRel)
if not silent:
    print(f"type4 = {len(wayRels)}")
allTypes.append(wayRels)

type4 = 121


## 5 - [Ganzjähriges Skibefahrungsverbot auf der Fläche außerhalb der Wege (Wegegebot)](https://wiki.openstreetmap.org/wiki/DE:Betretungsverbote_f%C3%BCr_Gebiete_im_Winter#Ganzj.C3.A4hriges_Skibefahrungsverbot_auf_der_Fl.C3.A4che_au.C3.9Ferhalb_der_Wege_.28Wegegebot.29)

In [7]:
wayRels = []
for wayRel in allResults:
    seasonal, ski, ski_conditional, access, access_conditional, access_offroad, access_offroad_conditional = get_tags(wayRel.tags)

    if ( 
        seasonal is None 
        or seasonal == 'no' 
    ) and (
        access_offroad in ['no','private'] and (
            ski is None
            or not ski in ['yes','designated','permissive']
        )
    ):
        wayRels.append(wayRel)
if not silent:
    print(f"type5 = {len(wayRels)}")
allTypes.append(wayRels)

type5 = 13


## 6 - [Monatsabhängiges oder saisonales Skibefahrungsverbot auf der Fläche außerhalb der Wege (saisonales Wegegebot)](https://wiki.openstreetmap.org/wiki/DE:Betretungsverbote_f%C3%BCr_Gebiete_im_Winter#Monatsabh.C3.A4ngiges_oder_saisonales_Skibefahrungsverbot_auf_der_Fl.C3.A4che_au.C3.9Ferhalb_der_Wege_.28saisonales_Wegegebot.29)

In [8]:
wayRels = []
for wayRel in allResults:
    seasonal, ski, ski_conditional, access, access_conditional, access_offroad, access_offroad_conditional = get_tags(wayRel.tags)
    if (
        (
            (
                not (seasonal is None or seasonal == 'no')
            ) and (
                access_offroad in ['no','private']
                or re.match('(no|private) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', access_offroad_conditional)
            ) and (
                ski is None
                or not ski in ('yes','designated','permissive')
            )
        )
    ) or (
        re.match('(no|private) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', access_offroad_conditional)
        and (
            ski is None
        )
    ):
        wayRels.append(wayRel)
if not silent:
    print(f"type6 = {len(wayRels)}")
allTypes.append(wayRels)

type6 = 3


## 7 - [Ganzjährige "Bitte um Nichtbefahrung" der Fläche außerhalb der Wege](https://wiki.openstreetmap.org/wiki/DE:Betretungsverbote_f%C3%BCr_Gebiete_im_Winter#Ganzj.C3.A4hrige_.22Bitte_um_Nichtbefahrung.22_der_Fl.C3.A4che_au.C3.9Ferhalb_der_Wege)

In [9]:
wayRels = []
for wayRel in allResults:
    seasonal, ski, ski_conditional, access, access_conditional, access_offroad, access_offroad_conditional = get_tags(wayRel.tags)

    if ( 
        seasonal is None 
        or seasonal == 'no'
    ) and (
        access_offroad == 'discouraged' and (
            ski is None
            or not ski in ['yes','designated','permissive']
        )
    ):
        wayRels.append(wayRel)
if not silent:
    print(f"type7 = {len(wayRels)}")
allTypes.append(wayRels)

type7 = 1


## 8 - [Monatsabhängige oder saisonale "Bitte um Nichtbefahrung" auf der Fläche außerhalb der Wege](https://wiki.openstreetmap.org/wiki/DE:Betretungsverbote_f%C3%BCr_Gebiete_im_Winter#Monatsabh.C3.A4ngige_oder_saisonale_.22Bitte_um_Nichtbefahrung.22_auf_der_Fl.C3.A4che_au.C3.9Ferhalb_der_Wege)

In [10]:
wayRels = []
for wayRel in allResults:
    seasonal, ski, ski_conditional, access, access_conditional, access_offroad, access_offroad_conditional = get_tags(wayRel.tags)

    if (
        (
            (
                not (seasonal is None or seasonal == 'no')
            ) and (
                access_offroad == 'discouraged' 
                or re.match('(discouraged) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', access_offroad_conditional)
            ) and (
                ski is None
                or not ski in ['yes','designated','permissive']
            )
        )
    ) or (
        re.match('(discouraged) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', access_offroad_conditional)
        and (
            ski is None
        )
    ):
        wayRels.append(wayRel)
if not silent:
    print(f"type8 = {len(wayRels)}")
allTypes.append(wayRels)

type8 = 0


## Tags af Basis der einzelnen Listen setzen

In [11]:
multiClassifieds = []
unclassifieds = []
typeCount = [0,0,0,0,0,0,0,0]

for wayRel in allResults:
    types = []
    for i in range(0, 7):
        if wayRel in allTypes[i]:
            types.append(f"type{i+1}")
            typeCount[i] += 1
    if len(types) == 1:
        wayRel.tags['classification'] = types[0]
    else:
        wayRel.tags['classification'] = ','.join(types)
    if type(wayRel) == overpy.Relation:
        wayRel.tags["@id"] = f"relation/{wayRel.id}"
    else:
        wayRel.tags["@id"] = f"way/{wayRel.id}"

    if len(types) > 1:
        multiClassifieds.append(wayRel)

    if len(types) == 0:
        unclassifieds.append(wayRel)

## Ergebnisse:

In [12]:
def href(id):
    return f"<a href='https://www.openstreetmap.org/way/{id}'>{id}</a>"

coverage = sum(typeCount) + len(unclassifieds)
if coverage != len(allResults):
    display(HTML(f"Coverage {coverage} != {len(allResults)} - only ok if Multiple Classifications != 0 !!!"))

html = f"<h3>Type overview [{sum(typeCount)}]</h3><ul>"
for i in range(0, 7):
    html += f"<li>{i+1}: {typeCount[i]}</li>"
display(HTML(f"{html}</u>"))

if len(multiClassifieds) > 0:
    html = f"<h3>Multiple Classifications [{len(multiClassifieds)}]</h3><ul>"
    for way in multiClassifieds:
        wayId = href(way.id)
        html += f"<li>{wayId} -> {way.tags['classification']}</li>"
    display(HTML(f"{html}</u>"))

html = f"<h3>Unclassified [{len(unclassifieds)}]</h3><ul>"
for way in unclassifieds:
    wayId = href(way.id)
    html += f"<li>{wayId} -> {way.tags}</li>"
display(HTML(f"{html}</u>"))

## Create a geojson with explicit tags for the 8 types
* tag of the property is s_type, values are type1 ... type8
* styling is still work in progress, for now, there's one single style for all types
  * for http://geojson.io this is limited by what styling capabilities are available there
  * TBD for leaflet (and a "reference" map)

In [13]:
import utils.ovp2geojson as o2p

# beware: changing this file requires a jupyter kernel restart!
o2p.create(result, allResults, f"data/Schongebiete.geojson")

Unsupported relation 10051540 of relations
Unsupported relation 10051540 of relations
Unsupported relation 10051540 of relations
Unsupported relation 11916966 of relations
